# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/savirahiman/Desktop/Git_Hub_Repo/python-api-challenge/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kudahuvadhoo,2.6708,72.8944,27.81,76,85,2.69,MV,1725308805
1,1,bethel,41.3712,-73.4140,24.74,59,18,0.89,US,1725308806
2,2,edinburgh of the seven seas,-37.0676,-12.3116,9.45,66,92,12.04,SH,1725308807
3,3,whitehorse,60.7161,-135.0538,15.61,54,75,2.57,CA,1725308809
4,4,mirriah,13.7073,9.1501,28.54,60,100,3.17,NE,1725308810


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=650,
    frame_height=500,
    size="Humidity",
    color="City"
    )

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=30)\
                            &(city_data_df['Max Temp']>=20)\
                            &(city_data_df['Cloudiness']<=90)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Humidity']<70),:]
print(len(ideal_df))

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

52


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,ribeira grande,38.5167,-28.7000,23.22,60,20,6.69,PT,1725308814
8,8,al jawf,29.5000,38.7500,27.19,26,0,4.36,SA,1725308815
14,14,ifrane,33.3333,-5.2500,22.87,48,0,1.05,MA,1725308822
31,31,mossel bay,-34.1831,22.1460,22.96,66,15,1.81,ZA,1725308843
41,41,leava,-14.3000,-178.1667,26.34,68,79,8.57,WF,1725308855


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ' '

# Display sample data
hotel_df.head()

/var/folders/8q/xhz52zqs5bj2xdbvnsfs5t980000gn/T/ipykernel_12279/3493997470.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ' '


,City,Country,Lat,Lng,Humidity,Hotel Name
7,ribeira grande,PT,38.5167,-28.7000,60,
8,al jawf,SA,29.5000,38.7500,26,
14,ifrane,MA,33.3333,-5.2500,48,
31,mossel bay,ZA,-34.1831,22.1460,66,
41,leava,WF,-14.3000,-178.1667,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "limit":10,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ribeira grande - nearest hotel: Hospedaria JSF
al jawf - nearest hotel: No hotel found
ifrane - nearest hotel: la perle d'azrou
mossel bay - nearest hotel: Protea Hotel Mossel Bay
leava - nearest hotel: Fia fia
muscat - nearest hotel: البطاشي
mto wa mbu - nearest hotel: Kirurumu Manyara Lodge
thompson - nearest hotel: Thompson Inn
kalemie - nearest hotel: Hotel du Lac
albufeira - nearest hotel: Hotel Baltum
mana - nearest hotel: Le Samana
zarghun shahr - nearest hotel: No hotel found
vorona teodoru - nearest hotel: No hotel found
zambezi - nearest hotel: No hotel found
kashgar - nearest hotel: 深航国际酒店
tinajo - nearest hotel: No hotel found
luganville - nearest hotel: Beachfront Resort
dalandzadgad - nearest hotel: Хан Уул буудал
farah - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
berrouaghia - nearest hotel: Andalous
mil'kovo - nearest hotel: Дрозды
kalabo - nearest hotel: Golden Lodge Luxury Acomodation
campbell river - nea

,City,Country,Lat,Lng,Humidity,Hotel Name
7,ribeira grande,PT,38.5167,-28.7000,60,Hospedaria JSF
8,al jawf,SA,29.5000,38.7500,26,No hotel found
14,ifrane,MA,33.3333,-5.2500,48,la perle d'azrou
31,mossel bay,ZA,-34.1831,22.1460,66,Protea Hotel Mossel Bay
41,leava,WF,-14.3000,-178.1667,68,Fia fia
52,muscat,OM,23.6139,58.5922,68,البطاشي
75,mto wa mbu,TZ,-3.3500,35.8500,67,Kirurumu Manyara Lodge
83,thompson,CA,55.7435,-97.8558,44,Thompson Inn
102,kalemie,CD,-5.9475,29.1947,60,Hotel du Lac
106,albufeira,PT,37.0882,-8.2503,68,Hotel Baltum


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=650,
    frame_height=500,
    size="Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"]
    )

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)